<a href="https://colab.research.google.com/github/11Alejandro/Trabajo-final---Parqueadero/blob/main/Gesti%C3%B3n_parqueadero3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =============================== LIBRERIAS Y RUTA =============================

import os
from datetime import datetime

ARCHIVO_DATOS = "parqueadero.txt"

usuarios = {}
vehiculos_en_el_parqueadero = {}
historial_retiros = []
usuarios_admin = {"admin": "admin123", "profesor": "profe456"}
tiempo_actuales = []

# ========================== FUNCIONES DE GUARDADO/CARGA =======================

def guardar_todo_en_un_archivo():
    with open(ARCHIVO_DATOS, "w") as f:
        f.write("# USUARIOS\n")
        for doc, data in usuarios.items():
            f.write(f"{doc},{data['nombre']},{data['apellido']},{data['placa']}\n")

        f.write("\n# VEHICULOS_PARQUEADOS\n")
        for doc, data in vehiculos_en_el_parqueadero.items():
            f.write(f"{doc},{data['hora_ingreso'].strftime('%Y-%m-%d %H:%M:%S')}\n")

        f.write("\n# HISTORIAL_RETIROS\n")
        for item in historial_retiros:
            f.write(f"{item['documento']},{item['tiempo_parqueado_minutos']},{item['total_pagado']}\n")

def cargar_todo_desde_un_archivo():
    global usuarios, vehiculos_en_el_parqueadero, historial_retiros
    usuarios = {}
    vehiculos_en_el_parqueadero = {}
    historial_retiros = []

    if not os.path.exists(ARCHIVO_DATOS):
        return

    with open(ARCHIVO_DATOS, "r") as f:
        seccion = None
        for linea in f:
            linea = linea.strip()
            if not linea or linea.startswith("#"):
                if "USUARIOS" in linea:
                    seccion = "usuarios"
                elif "VEHICULOS_PARQUEADOS" in linea:
                    seccion = "parqueados"
                elif "HISTORIAL_RETIROS" in linea:
                    seccion = "retiros"
                continue

            if seccion == "usuarios":
                doc, nombre, apellido, placa = linea.split(",")
                usuarios[doc] = {
                    "nombre": nombre,
                    "apellido": apellido,
                    "documento_identidad": doc,
                    "placa": placa
                }

            elif seccion == "parqueados":
                doc, hora_str = linea.split(",")
                hora_ingreso = datetime.strptime(hora_str, "%Y-%m-%d %H:%M:%S")
                vehiculos_en_el_parqueadero[doc] = {"hora_ingreso": hora_ingreso}

            elif seccion == "retiros":
                doc, minutos, total = linea.split(",")
                historial_retiros.append({
                    "documento": doc,
                    "tiempo_parqueado_minutos": int(minutos),
                    "total_pagado": int(total)
                })

# ======================== FUNCIONES DE VALIDACIÓN =============================

# Validación de nombres
def validar_nombre(nombre):
    if len(nombre) < 3:
        print("\033[91mEl nombre debe tener al menos 3 letras. Por favor, inténtelo otra vez.\033[0m")
        return False
    if not nombre.isalpha():
        print("\033[91mEl nombre solo debe contener letras sin espacios, números ni símbolos. Por favor, inténtelo otra vez.\033[0m")
        return False
    return True

# Validación de apellidos
def validar_apellido(apellido):
    if len(apellido) < 3:
        print("\033[91mEl apellido debe tener al menos 3 letras. Por favor, inténtelo otra vez.\033[0m")
        return False
    if not apellido.isalpha():
        print("\033[91mEl apellido solo debe contener letras sin espacios, números ni símbolos. Por favor, inténtelo otra vez.\033[0m")
        return False
    return True

# Validación del documento de identidad
def validar_documento(documento):
    if not documento.isdigit():
        print("\033[91mEl documento solo debe contener números. Por favor, inténtelo otra vez.\033[0m")
        return False
    if len(documento) < 3 or len(documento) > 15:
        print("\033[91mEl documento debe tener entre 3 y 15 dígitos. Por favor, inténtelo otra vez.\033[0m")
        return False
    return True

# Validación de la placa
def validar_placa(placa):
    placa = placa.upper()
    if len(placa) != 6:
        print("\033[91mLa placa debe tener exactamente 6 caracteres. Por favor, inténtelo otra vez.\033[0m")
        return False
    letras = placa[:3]
    numeros = placa[3:]
    if not letras.isalpha() or not numeros.isdigit():
        print("\033[91mLa placa debe tener 3 letras seguidas de 3 números. Ej: ABC123. Por favor, inténtelo otra vez.\033[0m")
        return False
    return True

# =============== FUNCIÓN Y PROCEDIMIENTO PARA REGISTRAR USUARIOS ==============

def registrar_nuevo_usuario():
    print("\n\033[1m------------------ REGISTRO DE NUEVO USUARIO --------------------\033[0m")

    while True:
        nombre = input("Ingrese su nombre: ")
        if validar_nombre(nombre):
            break

    while True:
        apellido = input("Ingrese su apellido: ")
        if validar_apellido(apellido):
            break

    while True:
        documento = input("Ingrese su número de documento: ")
        if validar_documento(documento):
            break

    while True:
        placa = input("Ingrese la placa del vehículo (formato ABC123): ")
        if validar_placa(placa):
            placa = placa.upper()
            break

    usuarios[documento] = {
        "nombre": nombre,
        "apellido": apellido,
        "documento_identidad": documento,
        "placa": placa
    }

    print("\033[1;32m------ El usuario ha sido registrado exitosamente ------\033[0m\n")


    print("\n\033[1;32m-------------- Resumen de ingreso --------------\033[0m")
    print(f"Documento: {documento}")
    print(f"Nombre: {nombre}")
    print(f"Apellido: {apellido}")
    print(f"Placa del vehículo: {placa}")
    print("\033[1;32m----------------------------------------------------\033[0m")

# ================ FUNCIÓN Y PROCEDIMIENTO PARA INGRESAR VEHICULO ==============

def ingresar_vehiculo():
    while True:
        print("\n\033[1m=============== INGRESO DE VEHÍCULO ===============\033[0m")

        documento = input("Ingrese el número de documento del usuario: ")

        if documento not in usuarios:
            print("\033[91mEste documento no está registrado.\033[0m")
            opcion = input("¿Desea registrar el usuario? (si/no):").lower()
            if opcion == "si":
                registrar_nuevo_usuario()
            else:
                print("\033[91mNo se puede continuar sin un usuario registrado.\033[0m")
                return

        if documento in vehiculos_en_el_parqueadero:
            print("\033[91mEste usuario ya tiene un vehículo dentro del parqueadero.\033[0m")
            return

        if len(vehiculos_en_el_parqueadero) >= 64:
            print("\033[91mEn este momento no hay celdas disponibles.\033[0m")
            return

        hora_ingreso = datetime.now()
        vehiculos_en_el_parqueadero[documento] = {"hora_ingreso": hora_ingreso}

        nombre = usuarios[documento]["nombre"]
        apellido = usuarios[documento]["apellido"]
        placa = usuarios[documento]["placa"]
        hora_ingreso = hora_ingreso.strftime("%Y-%m-%d %H:%M:%S")

        print("\n\033[1;32m-------------- Resumen de ingreso --------------\033[0m")
        print(f"\033[1mDocumento:\033[0m {documento}")
        print(f"\033[1mNombre:\033[0m {nombre}")
        print(f"\033[1mApellido:\033[0m {apellido}")
        print(f"\033[1mPlaca del vehículo:\033[0m {placa}")
        print(f"\033[1mHora de ingreso:\033[0m {hora_ingreso}")
        print("\033[1;32m----------------------------------------------------\033[0m")



# =============== FUNCIÓN Y PROCEDIMIENTO PARA RETIRAR VEHICULO ================

def retirar_vehiculo():
    print("\n\033[1m=============== RETIRO DE VEHÍCULO ===============\033[0m")

    usuario = input("Ingrese el número de documento del usuario: ")

    if usuario in usuarios:
        print("Usuario verificado correctamente. Puede retirar su vehículo.")
    else:
        print("No hay ningún vehículo registrado para este documento.")
        opcion = input("¿Desea registrarse? (si/no): ").lower()
        if opcion == "si":
            registrar_nuevo_usuario()
        return

    try:
        tiempo_parqueado = int(input("Ingrese el tiempo que el vehículo estuvo parqueado (en minutos): "))
    except ValueError:
        print("Entrada inválida. Debe ingresar un número entero.")
        return

    valor_hora = 7000
    valor_cuarto = 1500

    horas = tiempo_parqueado // 60
    minutos_restantes = tiempo_parqueado % 60

    if minutos_restantes == 0:
        cuartos = 0
    elif minutos_restantes <= 15:
        cuartos = 1
    elif minutos_restantes <= 30:
        cuartos = 2
    elif minutos_restantes <= 45:
        cuartos = 3
    else:
        cuartos = 4

    total_horas = horas * valor_hora
    total_cuartos = cuartos * valor_cuarto
    total_a_pagar = total_horas + total_cuartos

    if total_a_pagar < valor_hora:
        total_a_pagar = valor_hora
        print("Se aplicó el pago mínimo de 7000 pesos.")

    print("Horas completas:", horas)
    print("Minutos restantes:", minutos_restantes)
    print("Cuartos de hora a cobrar:", cuartos)
    print("Cobro por horas:", total_horas)
    print("Cobro por cuartos:", total_cuartos)
    print("Total a pagar por el parqueo:", total_a_pagar, "pesos")


# ==================== ACCESO AL MÓDULO DE ADMINISTRACIÓN ======================

# ---------------------- Módulo de administración ------------------------------

def iniciar_sesion_admin():
    print("\n\033[1m=============== MÓDULO DE ADMINISTRACIÓN/REPORTES ===============\033[0m")

    usuario = input("Ingrese su usuario: ")
    contrasena = input("Ingrese su contraseña: ")
    if usuario in usuarios_admin and usuarios_admin[usuario] == contrasena:
        print("Bienvenido al menú del módulo de administración.")
        return True
    else:
        print("Usuario o contraseña incorrectos.")
        return False
# --------------------- Reporte de vehículos registrados -----------------------

def generar_reporte_total_vehiculos_registrados():
    print("\n--- Reporte: Total de vehículos registrados ---")
    total = len(usuarios)
    print(f"Total de usuarios con vehículo registrado: {total}.")

# --------------------- Reporte de vehículos retirados -------------------------

def generar_reporte_total_vehiculos_retirados():
    print("\n--- Reporte: Total de vehículos retirados ---")
    total = len(historial_retiros)
    print(f"Total de vehículos que han sido retirados: {total}.")

# --------------------- Reporte de vehículos sin retirar -----------------------

def generar_reporte_total_vehiculos_sin_retirar():
    print("\n--- Reporte: Total de vehículos sin retirar ---")
    total = len(vehiculos_en_el_parqueadero)
    print(f"Total de vehículos actualmente en el parqueadero: {total}.")

# ------------------- Reporte de pago de vehículos retirados -------------------

def generar_reporte_total_pago_vehiculos_retirados():
    print("\n--- Reporte: Total pago de vehículos retirados ---")
    total_pagado = sum(item['total_pagado'] for item in historial_retiros)
    print(f"Monto total recaudado por vehículos retirados: ${total_pagado:,.2f} pesos.")

# --------------------- Reporte de promedio de estancia ------------------------

def generar_reporte_tiempo_promedio_estancia():
    print("\n--- Reporte: Tiempo promedio de estancia por vehículo (basado en historial) ---")
    if not historial_retiros:
        print("No hay vehículos retirados para calcular el promedio de estancia.")
        return

    total_minutos = sum(item['tiempo_parqueado_minutos'] for item in historial_retiros)
    promedio_minutos = total_minutos / len(historial_retiros)
    promedio_horas = promedio_minutos / 60
    print(f"El tiempo promedio de estancia de los vehículos retirados es de {promedio_minutos:.2f} minutos ({promedio_horas:.2f} horas).")

# --------------------- Reporte de usuarios registrados ------------------------

def generar_reporte_lista_usuarios():
    print("\n--- Reporte: Lista de usuarios registrados ---")
    if not usuarios:
        print("No hay usuarios registrados en el sistema.")
        return
    for doc, data in usuarios.items():
        print(f"Documento: {doc}, Nombre: {data['nombre']} {data['apellido']}, Placa: {data['placa']}")

# --------------- Reporte de tiempo máximo y mínimo de parqueo -----------------

def generar_reporte_vehiculo_tiempo_maximo_minimo_parqueo():
    print("\n--- Reporte: Vehículo con tiempo de parqueo máximo y mínimo (actualmente parqueados) ---")
    if not vehiculos_en_el_parqueadero:
        print("No hay vehículos actualmente en el parqueadero para calcular tiempos.")
        return

    ahora = datetime.now()

    for doc, data in vehiculos_en_el_parqueadero.items():
        tiempo_delta = ahora - data["hora_ingreso"]
        minutos_actuales = int(tiempo_delta.total_seconds() / 60)
        placa = data.get("placa", "N/A") # Asegurarse de tener la placa
        tiempos_actuales.append({"documento": doc, "placa": placa, "minutos": minutos_actuales})

    if not tiempos_actuales:
        print("No hay vehículos con tiempo de parqueo computable en este momento.")
        return
# -------------------- Encontrar el vehículo con más tiempo --------------------

    vehiculo_max_tiempo = max(tiempos_actuales, key=lambda x: x['minutos'])
    print(f"Vehículo con más tiempo parqueado (actualmente):")
    print(f"Placa: {vehiculo_max_tiempo['placa']}, Documento: {vehiculo_max_tiempo['documento']}")
    print(f"Tiempo: {vehiculo_max_tiempo['minutos']} minutos")

# -------------------- Encontrar el vehículo con menos tiempo ------------------

    vehiculo_min_tiempo = min(tiempos_actuales, key=lambda x: x['minutos'])
    print(f"Vehículo con menos tiempo parqueado (actualmente):")
    print(f"Placa: {vehiculo_min_tiempo['placa']}, Documento: {vehiculo_min_tiempo['documento']}")
    print(f"Tiempo: {vehiculo_min_tiempo['minutos']} minutos")

# ----------------------- Menú del módulo de administración --------------------

def menu_administracion():
    while True:
        print("\n\033[1m================ Menú de Administración ================\033[0m")

        print("1. Total de vehículos registrados (usuarios)")
        print("2. Total de vehículos retirados (historial)")
        print("3. Total de vehículos sin retirar (actualmente en parqueadero)")
        print("4. Total pago de vehículos retirados")
        print("5. Tiempo promedio de estancia por vehículo en el parqueadero")
        print("6. Lista de usuarios registrados")
        print("7. Vehículo con tiempo de parqueo máximo y mínimo (actuales)")
        print("8. Salir del menú de administración")
        opcion = input("Seleccione una opción (1-8): ")

        if opcion == "1":
            generar_reporte_total_vehiculos_registrados()
        elif opcion == "2":
            generar_reporte_total_vehiculos_retirados()
        elif opcion == "3":
            generar_reporte_total_vehiculos_sin_retirar()
        elif opcion == "4":
            generar_reporte_total_pago_vehiculos_retirados()
        elif opcion == "5":
            generar_reporte_tiempo_promedio_estancia()
        elif opcion == "6":
            generar_reporte_lista_usuarios()
        elif opcion == "7":
            generar_reporte_vehiculo_tiempo_maximo_minimo_parqueo()
        elif opcion == "8":
            print("Saliste del módulo de administración. ¡Hasta pronto!")
            break
        else:
            print("Opción inválida. Intente de nuevo.")


# ============================= MENÚ PRINCIPAL =================================

def menu_parqueadero():
    while True:
        print(r"""
                            ______
                           /|_||_\`.__
                          (   _    _ _\
                          =`-(_)--(_)-'

         """ + "\033[1m" + "       ¡BIENVENIDO AL PARQUEADER0 AUTOPARK!" + "\033[0m")

        print("\033[1m======================== MENÚ PRINCIPAL ========================\033[0m")
        print("1. Registrar usuario")
        print("2. Ingreso del vehículo")
        print("3. Retiro del vehículo")
        print("4. Módulo de administración/reportes")
        print("5. Salir")
        opcion = input("Selecciona una opción (1-5): ")

        if opcion == "1":
            registrar_nuevo_usuario()
            guardar_todo_en_un_archivo()
        elif opcion == "2":
            ingresar_vehiculo()
            guardar_todo_en_un_archivo()
        elif opcion == "3":
            retirar_vehiculo()
            guardar_todo_en_un_archivo()
        elif opcion == "4":
            if iniciar_sesion_admin():
                menu_administracion()
        elif opcion == "5":
            print("Saliste del menú principal. ¡Hasta pronto!")
            guardar_todo_en_un_archivo()
            break
        else:
            print("Opción inválida. Intente de nuevo.")

# ========================== INICIO DEL PROGRAMA ===============================

cargar_todo_desde_un_archivo()
menu_parqueadero()



                            ______
                           /|_||_\`.__
                          (   _    _ _\
                          =`-(_)--(_)-'

                ¡BIENVENIDO AL PARQUEADER0 AUTOPARK!
======================== MENÚ PRINCIPAL ========================
1. Registrar usuario
2. Ingreso del vehículo
3. Retiro del vehículo
4. Módulo de administración/reportes
5. Salir

=============== MÓDULO DE ADMINISTRACIÓN/REPORTES ===============
Bienvenido al menú del módulo de administración.

================ Menú de Administración ================
1. Total de vehículos registrados (usuarios)
2. Total de vehículos retirados (historial)
3. Total de vehículos sin retirar (actualmente en parqueadero)
4. Total pago de vehículos retirados
5. Tiempo promedio de estancia por vehículo en el parqueadero
6. Lista de usuarios registrados
7. Vehículo con tiempo de parqueo máximo y mínimo (actuales)
8. Salir del menú de administración

--- Reporte: Total de vehículos registrados ---
Total de